In [1]:
from ase.io import read, write
from ase.neighborlist import NeighborList, natural_cutoffs
from ase.data import covalent_radii, atomic_numbers
from ase import Atom
import numpy as np
from ase.visualize import view

In [2]:
cluster = read('LMS CONTCAR Exp.xyz', format='xyz')

In [3]:
scaling_factor = 0.85
cutoffs = natural_cutoffs(cluster, mult=scaling_factor)

In [4]:
nl = NeighborList(cutoffs, self_interaction=False, bothways=True)
nl.update(cluster)

True

In [5]:
atom_indices_to_check = [287,288, 295]  # Replace with indices relevant to your cluster

for atom_index in atom_indices_to_check:
    indices, offsets = nl.get_neighbors(atom_index)
    print(f"\nAtom {atom_index} ({cluster[atom_index].symbol}) has neighbors:")
    for neighbor_index in indices:
        neighbor_atom = cluster[neighbor_index]
        distance = cluster.get_distance(atom_index, neighbor_index)
        print(f" - Neighbor {neighbor_index}: {neighbor_atom.symbol}, Distance: {distance:.2f} Å")


Atom 287 (O) has neighbors:
 - Neighbor 95: Li, Distance: 2.18 Å
 - Neighbor 139: Si, Distance: 1.70 Å
 - Neighbor 175: Si, Distance: 1.70 Å

Atom 288 (O) has neighbors:
 - Neighbor 16: Li, Distance: 1.97 Å

Atom 295 (O) has neighbors:
 - Neighbor 6: Li, Distance: 1.95 Å
 - Neighbor 23: Li, Distance: 1.97 Å
 - Neighbor 62: Li, Distance: 1.94 Å
 - Neighbor 198: Si, Distance: 1.61 Å


In [6]:
expected_coordination = {'Li': 4, 'Si': 4, 'O': 4}

In [7]:
undercoordinated_oxygens = []

for i, atom in enumerate(cluster):
    if atom.symbol == 'O':
        indices, offsets = nl.get_neighbors(i)
        actual_cn = len(indices)
        expected_cn = expected_coordination['O']
        if actual_cn < expected_cn:
            missing_bonds = expected_cn - actual_cn
            undercoordinated_oxygens.append((i, missing_bonds))

In [11]:
from scipy.spatial.transform import Rotation

def get_tetrahedral_vectors():
    """Returns four unit vectors pointing from the center to the vertices of a regular tetrahedron."""
    a = 1.0 / np.sqrt(3)
    return np.array([
        [ a,  a,  a],
        [-a, -a,  a],
        [-a,  a, -a],
        [ a, -a, -a],
    ])

oh_bond_length = 0.96

for index, missing_bonds in undercoordinated_oxygens:
    atom = cluster[index]
    atom_position = atom.position
    indices, offsets = nl.get_neighbors(index)
    neighbor_positions = cluster.positions[indices] + np.dot(offsets, cluster.get_cell())
    
    existing_bonds = neighbor_positions - atom_position
    existing_bonds /= np.linalg.norm(existing_bonds, axis=1)[:, np.newaxis]
    
    num_existing_bonds = len(existing_bonds)
    num_missing_bonds = missing_bonds

    ideal_bonds = get_tetrahedral_vectors()

    if num_existing_bonds >= 2:
        # Use Kabsch algorithm to find rotation matrix
        ideal_existing_bonds = ideal_bonds[:num_existing_bonds]
        
        # Apply Kabsch algorithm without transposing
        def kabsch(P, Q):
            """Finds the optimal rotation matrix R that minimizes the RMSD between P and Q."""
            P_mean = np.mean(P, axis=0)
            Q_mean = np.mean(Q, axis=0)
            P_centered = P - P_mean
            Q_centered = Q - Q_mean
            C = np.dot(P_centered.T, Q_centered)
            V, S, Wt = np.linalg.svd(C)
            d = np.linalg.det(np.dot(V, Wt)) < 0.0
            if d:
                Wt[-1, :] *= -1
            R = np.dot(V, Wt)
            return R

        R = kabsch(existing_bonds, ideal_existing_bonds)

        # Rotate all ideal bonds using the rotation matrix
        rotated_ideal_bonds = np.dot(ideal_bonds, R)

        # Identify missing bond directions
        used_indices = []
        for i in range(num_existing_bonds):
            distances = np.linalg.norm(rotated_ideal_bonds - existing_bonds[i], axis=1)
            min_index = np.argmin(distances)
            used_indices.append(min_index)
        
        missing_indices = [i for i in range(4) if i not in used_indices]
        missing_bond_vectors = rotated_ideal_bonds[missing_indices]
    elif num_existing_bonds == 1:
        # Handle single existing bond case
        existing_vec = existing_bonds[0]
        # Generate two vectors orthogonal to existing_vec
        arbitrary_vec = np.array([1, 0, 0])
        if np.allclose(existing_vec, arbitrary_vec):
            arbitrary_vec = np.array([0, 1, 0])
        ortho_vec1 = np.cross(existing_vec, arbitrary_vec)
        ortho_vec1 /= np.linalg.norm(ortho_vec1)
        ortho_vec2 = np.cross(existing_vec, ortho_vec1)
        ortho_vec2 /= np.linalg.norm(ortho_vec2)
        # Calculate missing bond vectors
        angle = 109.5 * np.pi / 180  # Tetrahedral angle in radians
        sin_theta = np.sin(angle)
        cos_theta = np.cos(angle)
        missing_bond_vectors = np.array([
            cos_theta * existing_vec + sin_theta * ortho_vec1,
            cos_theta * existing_vec - sin_theta * ortho_vec1,
            -existing_vec
        ])
    else:
        # No existing bonds; use ideal tetrahedral vectors
        missing_bond_vectors = ideal_bonds

    # Add hydrogens along missing bond vectors
    for vec in missing_bond_vectors:
        h_position = atom_position + vec * oh_bond_length
        cluster.append(Atom('H', position=h_position))

In [9]:
view(cluster)

<Popen: returncode: None args: ['/Library/Frameworks/Python.framework/Versio...>

In [10]:
write("updated_cluster.xyz", cluster)